In [ ]:
!pip install langchain transformers
!pip install langchain_community
!pip install langchain_core
!pip install langchain_together
!pip install streamlit
!pip install langchain-huggingface
!pip install pypdf
!pip install langchain-pinecone

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
from uuid import uuid4

pinecone_api_key=userdata.get('pinecone_api_key')
loader = PyPDFLoader("Indian Penal Code Book.pdf")
documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1",model_kwargs={"trust_remote_code":True,"revision":"cc62377b015c53a3bf52bb2f4eb8c55326d3f162"})

pc = Pinecone(api_key=pinecone_api_key)
index_name="legalaivectors"
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)
uuids = [str(uuid4()) for _ in range(len(texts))]
vector_store.add_documents(documents=texts, ids=uuids)
